### Como utilizar ML com Dados Não Estruturais Textuais?

* Técnicas de NLP preprocessam os dados para filtrarmos o que é mais relevante para a classificação 

![Title](../imgs/nlp-illustration.png)


* A tokenização nos auxilia a dividir strings em palavras
* Essas palavras serão as "features" que serão utilizadas para aprendizado de um modelo de Machine Learning
* Um algoritmo de Machine Learning só entende números (atributos sempre numéricos), sendo assim, após a tokenização, precisamos converter nossas palavras para valores numéricos
* O processo de transformação textual para base numérico denomina-se **bag of words**

![Title](../imgs/1-Bag-of-words.png)

* Técnicas como TF-IDF e CountVectorizer, apesar de eficientes, transformam o dataset em dado tabular
  * Cada coluna é uma palavra
  * Quando a palavra existir em uma dada amostra, substituimos o valor 0 pela frequência em que aquela palavra ocorre no texto
  
  <img src="https://www.researchgate.net/profile/Haider-Al-Khateeb/publication/291950178/figure/fig1/AS:330186932408324@1455734107458/Term-Frequency-Inverse-Document-Frequency-TF-IDF.png" width=800>
  
* A consequencia disso é a perda da relação semântica nas frases
* Perda de contexto das sentenças
* Vocabulário imenso (maldição da dimensionalidade)

### Word Embeddings to rescue!

* Vetor multidimensional de palavras
* Mapa de características com a frequência de palavras e considerações semânticas 
* Baseado em similaridade
* Aprendizado por meio de redes neurais

##### Como fazemos isso?
* Dada uma palavra da frase
* Pegar as palavras vizinhas 
* Rede Neural irá calcular a probabilidade dessa palavra vizinha ser a palavra a ser predita
* Selecionamos as palavras vizinhas definindo uma "janela" ou "kernel"
  * Janela = 2 considere duas palavras ao redor da palavra atual
![Title](../imgs/training_data.png)

* Criamos um corpus com as palavras organizadas de acordo com as janelas em torno do dataset.
* Palavras que aparecerem juntas tem maior probabilidade de terem significado semântico
  * Exemplo: "soviética" provavelmente terá uma semelhança com união
  
 ![Title](../imgs/vetor_multi.png)
 
[Leitura adicional](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/): funcionamento de uma rede neural para embeddings

### Arquiteturas Word Embeddings

* Existem diferentes formas de gerar nossas embeddings, gerando determinadas arquiteturas previamente conhecidos como CBOW e SkipGram. Essas arquiteturas também são denominadas como tipo word2vec (palavra para vetor)

* CBOW: predizer uma palavra baseado nas palavras vizinhas. Mais rápido, funciona melhor nas palavras mais frequentes.
* Skip-Gram: predizer o contexto baseado nas palavras vizinhas. Funciona melhor com datasets menores
<img src="https://leimao.github.io/images/article/2019-08-23-Word2Vec-Classic/word2vec.png">
* Ambas arquiteturas tem uma ambientação semelhante, modificando a forma como a saída e a entrada são organizadas
* Exemplo: Hoje vai fazer sol pela manhã com pancadas de chuva à tarde
* Utilizando CBOW
![Title](../imgs/cbow.png)
* Utilizando Skip-Gram
![Title](../imgs/skip_gram.png)

[Leitura adicional](https://www.tensorflow.org/tutorials/text/word2vec): transformando o corpus do CBOW/skip_gram em numérico

In [ ]:
!pip install gensim

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Baixa as listas de stopwords e as tokenizações
nltk.download('stopwords')
nltk.download('punkt')

# Define as stopwords em inglês
sw_english = set(stopwords.words('english'))

# Instância o PorterStemmer
stemmer = PorterStemmer()

# Carrega o conjunto de dados
movies = pd.read_csv('../datasets/movies.csv', index_col = 0)

# Retira uma amostra do conjunto de dados
movies_sample = movies.sample(frac = 0.1, replace=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cecilia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/cecilia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# Cria um pipeline
def preprocessing(string):
    ###
    # Deixa apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", ' ', string)
    ###
    # deixa todas as palavras minúsculas
    string = string.lower()
    ###
    # tokenização
    words = word_tokenize(string)
    ###
    # Remove Stopwords
    filtered_words = []
    for w in words:
        if w not in sw_english:
            filtered_words.append(w)
    ###
    # Aplica o Stemming
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    # Retorna a lista de palavras pré-processadas
    return stem_words

# Aplica o preprocessing nas críticas de filmes
movies_sample["filtered_words"] = movies_sample['text'].apply(lambda x: preprocessing(x))

In [36]:
movies_sample

,text,label,filtered_words
36151,Well let me just say something about these act...,1,"[well, let, say, someth, actor, realli, good, ..."
31632,Now we know where they got the idea of Snakes ...,0,"[know, got, idea, snake, plane, put, bluntli, ..."
13602,"This was a great movie, and safe for the entir...",1,"[great, movi, safe, entir, famili, one, see, m..."
22447,Myron Breckinridge (Rex Reed!!!) gets a sex ch...,0,"[myron, breckinridg, rex, reed, get, sex, chan..."
36511,I absolutely despise this movie. No wonder Jos...,0,"[absolut, despis, movi, wonder, jose, larraz, ..."
...,...,...,...
27014,"despite the occasionally stilted acting and ""s...",1,"[despit, occasion, stilt, act, seen, stori, fa..."
2802,I like a lot of Myrna Loy movies. This film wa...,0,"[like, lot, myrna, loy, movi, film, produc, ch..."
13917,This film quite literally has every single act...,1,"[film, quit, liter, everi, singl, action, movi..."
3696,This movie is without a doubt the worst horror...,0,"[movi, without, doubt, worst, horror, movi, ev..."


In [17]:
    
# Define e treina o modelo
model = Word2Vec(movies_sample["filtered_words"], min_count = 2)
model = Word2Vec(
    sentences=movies_sample["filtered_words"], # Passando a lista de tokens (List[List[str]])
    vector_size=100,   # Tamanho do vetor de saída
    min_count=1,  # Filtro palavras raras
    workers=2,  # Multicore
    window=5, # Janela de palavras, nesse caso 5 palavras
    sg=1,  # 1 para skip-grams, 0 para o CBOW
)

In [20]:
words = list(model.wv.index_to_key)

In [21]:
# Mostra as primeiras 10 palavras do vocabulário
print("Primeiras 10 palavras: ", words[:10], '\n\n')

# a partir da palavra time, determinar as mais similares a ela segundo os textos
print('Palavras mais similares a time: \n', model.wv.most_similar('time'), '\n\n')

# Determinando o mapa característico para a palavra time
print('Vetor para a palavra time: \n', model.wv.get_vector('time', norm = True), '\n\n')

Primeiras 10 palavras:  ['br', 'movi', 'film', 'one', 'like', 'time', 'good', 'make', 'see', 'charact'] 


Palavras mais similares a time: 
 [('occas', 0.7878702878952026), ('regret', 0.7604853510856628), ('nineti', 0.7588347792625427), ('odd', 0.7531259655952454), ('harder', 0.7500264048576355), ('dozen', 0.7498865723609924), ('preciou', 0.749779462814331), ('dreck', 0.7448720932006836), ('anticip', 0.7436569929122925), ('categori', 0.7422799468040466)] 


Vetor para a palavra time: 
 [-0.07086917 -0.02880108  0.10448317 -0.05086483 -0.11392558 -0.28411302
  0.09523632  0.24049914 -0.02056965 -0.16064933 -0.00274689  0.02376291
 -0.08728712 -0.01855629  0.03074314 -0.10383829  0.05066813 -0.05032621
 -0.09702493 -0.24207158  0.00690291  0.01385544  0.18003273  0.00049998
 -0.06703884 -0.0799669   0.00422336 -0.01062266 -0.16542329  0.04170448
  0.07149941 -0.02969232  0.02628511 -0.2074668  -0.02223976  0.1612775
  0.00636118 -0.01654712  0.09820938 -0.09244086  0.16032499 -0.10601716

In [22]:
# transforma os vetores do word2vec para uma base X treinável
X = model.wv[words]

# Mostra a base
print('Base X treinável: \n', X)

Base X treinável: 
 [[ 0.05439223  0.04718336  0.47860208 ... -0.25150117  0.11204524
   0.10274327]
 [-0.30462608  0.27046406  0.34967715 ... -0.09623296 -0.15711568
  -0.14566195]
 [-0.245333    0.1662296   0.29211605 ... -0.12276348  0.09712406
   0.01893252]
 ...
 [-0.1259432   0.07642938  0.04828117 ... -0.10612808  0.03402569
   0.05972762]
 [-0.12425543  0.07222645  0.04515435 ... -0.09308802  0.02037374
   0.03676774]
 [-0.12848416  0.07137331  0.04459506 ... -0.10968777  0.02961594
   0.05630128]]


In [26]:
model.wv.similarity('peopl', 'like')

0.55063856

### Treinamento de ML

In [37]:
# Carrega o train_test_split para separar a base em treino e teste
from sklearn.model_selection import train_test_split

# Separa os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(movies_sample["filtered_words"],
                                                     movies_sample['label'],
                                                     test_size = 0.3,
                                                     random_state = 42)

In [39]:
from gensim.models import Word2Vec

# Define e treina o modelo
model = Word2Vec(X_train, min_count = 2)
#model = Word2Vec(
#    sentences=movies_sample["filtered_words"], # Passando a lista de tokens (List[List[str]])
#    vector_size=100,   # Tamanho do vetor de saída
#    min_count=1,  # Filtro palavras raras
#    workers=2,  # Multicore
#    window=5, # Janela de palavras, nesse caso 5 palavras
#    sg=1,  # 1 para skip-grams, 0 para o CBOW
#)

In [43]:
words = set(model.wv.index_to_key )
X_train_vect = np.array([np.array([model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([model.wv[i] for i in ls if i in words])
                         for ls in X_test])

/var/folders/t4/7csz5lt17fs3dd7hl2vvgns00000gn/T/ipykernel_67780/990797624.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([model.wv[i] for i in ls if i in words])
/var/folders/t4/7csz5lt17fs3dd7hl2vvgns00000gn/T/ipykernel_67780/990797624.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([model.wv[i] for i in ls if i in words])


In [ ]:
# Instanciando o modelo
model = RandomForestClassifier()

# treinamento do modelo
model.fit(X_train_vect, y_train)

# Gerando as predições
y_pred = model.predict(X_test_vect)

# Print do classification report
print(classification_report(y_test, y_pred))